convert

In [ ]:
import os
import wfdb
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

# Define the path to your original dataset directory (.dat and .hea files)
DATA_DIR = "C:/Users/M2-Winterfell/Downloads/electrocardiography-dataset-1.0.3/records500/02000"

# Define the destination directory for the converted .asc files
DEST_DIR = "C:/Users/M2-Winterfell/Downloads/converted_asc"
if not os.path.exists(DEST_DIR):
    os.makedirs(DEST_DIR)

# Function to convert a single record into .asc format without modifying its content
def convert_ecg_record_to_asc(file_path, out_filepath):
    try:
        # Load the record using wfdb (the file_path should be provided without extension)
        record = wfdb.rdrecord(file_path)
        # Save the complete signal data (all channels) into an .asc file
        np.savetxt(out_filepath, record.p_signal, fmt="%.6f")
    except Exception as e:
        print(f"Error converting {file_path}: {e}")

# Function to convert all ECG records in the folder to .asc format
def convert_all_ecg_to_asc(data_dir, dest_dir):
    for root, dirs, files in os.walk(data_dir):
        for file in tqdm(files, desc="Converting ECG files"):
            if file.endswith("_hr.dat"):
                # Construct the base filename (removing the '_hr.dat' extension)
                base_filename = file.replace("_hr.dat", "_hr")
                record_path = os.path.join(root, base_filename)
                # Define the output .asc file path
                out_filename = base_filename + ".asc"
                out_filepath = os.path.join(dest_dir, out_filename)
                convert_ecg_record_to_asc(record_path, out_filepath)

# Convert all records in the DATA_DIR to .asc format
convert_all_ecg_to_asc(DATA_DIR, DEST_DIR)